# MODELIZACIÓN PARA CLASIFICACIÓN

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier

#Desactivar la notación científica
pd.options.display.float_format = '{:.2f}'.format


Sustituir la ruta del proyecto.

In [2]:
directorio = r"C:\Users\DataScience\ONLINE_DS_THEBRIDGE_DaniCastillo\ML_LEAD SCORING\01_src\01_data_sample\TRABAJO"

Nombres de los ficheros de datos.

In [3]:
nombre_x = 'x_preseleccionado.pickle'
nombre_y = 'y_preseleccionado.pickle'


Cargar los datos.

In [4]:
# Crear la ruta correctamente
ruta_x= os.path.join(directorio, nombre_x)
ruta_y= os.path.join(directorio, nombre_y)


# Cargar el archivo pickle
x = pd.read_pickle(ruta_x)
y = pd.read_pickle(ruta_y)

print("Archivo cargado correctamente.")

Archivo cargado correctamente.


In [5]:
x

,tiempo_en_site_mms,score_actividad_mms,ult_actividad_SMS Sent,visitas_total_mms,paginas_vistas_mms,score_perfil_mms,ocupacion_Working Professional,ambito_Select,ult_actividad_Chat Conversation,ult_actividad_Converted to Lead,ocupacion_Unemployed,ult_actividad_Page Visited on Website,origen_Lead Add Form,fuente_Direct Traffic
id,,,,,,,,,,,,,,
660737,0.00,0.70,0.00,0.00,0.00,0.44,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
660728,0.30,0.70,0.00,0.10,0.12,0.44,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00
660727,0.68,0.60,0.00,0.04,0.10,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
660719,0.14,0.50,0.00,0.02,0.05,0.67,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
660681,0.63,0.70,0.00,0.04,0.05,0.78,0.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579622,0.63,0.60,1.00,0.26,0.13,0.56,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
579615,0.09,0.60,1.00,0.10,0.12,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
579564,0.82,0.70,0.00,0.16,0.13,0.67,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00


## MODELIZAR

### Reservar el dataset de validacion

In [6]:
train_x,val_x,train_y,val_y = train_test_split(x,y,test_size=0.3)

### Crear el pipe y el diccionario de algorimos, parámetros y valores a testar

In [7]:
pipe = Pipeline([('algoritmo',RandomForestClassifier())])

In [8]:
grid = [ {'algoritmo': [XGBClassifier()],
         'algoritmo__n_jobs': [-1],
         'algoritmo__learning_rate': [0.01,0.025,0.05,0.1],
         'algoritmo__max_depth': [5,10,20],
         'algoritmo__reg_alpha': [0,0.1,0.5,1],
         'algoritmo__reg_lambda': [0.01,0.1,1],
         'algoritmo__n_estimators': [100,500,1000]},
        ]

### Optimizar los hiper parámetros

Elegir si se quiere usar grid search o random search.

Comentar la opción que no se vaya a usar.

####  Con grid search

In [9]:
# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=grid,
    scoring='roc_auc',  
    cv=5,  # Validación cruzada con 5 folds
    verbose=0,
    n_jobs=-1  # Usa todos los núcleos disponibles
)


In [10]:

 modelo = grid_search.fit(train_x,train_y)

 pd.DataFrame(grid_search.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algoritmo,param_algoritmo__learning_rate,param_algoritmo__max_depth,param_algoritmo__n_estimators,param_algoritmo__n_jobs,param_algoritmo__reg_alpha,param_algoritmo__reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
26,1.31,0.09,0.03,0.00,"XGBClassifier(base_score=None, booster=None, c...",0.01,5,1000,-1,0.00,1.00,"{'algoritmo': XGBClassifier(base_score=None, b...",0.91,0.90,0.89,0.91,0.91,0.91,0.01,1
28,1.34,0.09,0.04,0.00,"XGBClassifier(base_score=None, booster=None, c...",0.01,5,1000,-1,0.10,0.10,"{'algoritmo': XGBClassifier(base_score=None, b...",0.91,0.91,0.89,0.91,0.91,0.91,0.01,2
329,0.24,0.08,0.01,0.00,"XGBClassifier(base_score=None, booster=None, c...",0.10,5,100,-1,0.10,1.00,"{'algoritmo': XGBClassifier(base_score=None, b...",0.91,0.91,0.89,0.91,0.91,0.91,0.01,3
31,1.39,0.19,0.04,0.00,"XGBClassifier(base_score=None, booster=None, c...",0.01,5,1000,-1,0.50,0.10,"{'algoritmo': XGBClassifier(base_score=None, b...",0.91,0.90,0.89,0.91,0.91,0.91,0.01,4
30,1.53,0.11,0.05,0.01,"XGBClassifier(base_score=None, booster=None, c...",0.01,5,1000,-1,0.50,0.01,"{'algoritmo': XGBClassifier(base_score=None, b...",0.91,0.90,0.89,0.91,0.91,0.91,0.01,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,3.24,0.23,0.12,0.01,"XGBClassifier(base_score=None, booster=None, c...",0.10,20,1000,-1,0.00,0.01,"{'algoritmo': XGBClassifier(base_score=None, b...",0.89,0.87,0.85,0.89,0.88,0.88,0.01,428
76,1.70,0.33,0.04,0.01,"XGBClassifier(base_score=None, booster=None, c...",0.01,20,100,-1,0.10,0.10,"{'algoritmo': XGBClassifier(base_score=None, b...",0.90,0.86,0.87,0.88,0.87,0.88,0.01,429
73,1.36,0.21,0.05,0.00,"XGBClassifier(base_score=None, booster=None, c...",0.01,20,100,-1,0.00,0.10,"{'algoritmo': XGBClassifier(base_score=None, b...",0.89,0.86,0.87,0.88,0.87,0.88,0.01,430
75,1.43,0.14,0.06,0.02,"XGBClassifier(base_score=None, booster=None, c...",0.01,20,100,-1,0.10,0.01,"{'algoritmo': XGBClassifier(base_score=None, b...",0.89,0.86,0.87,0.88,0.87,0.87,0.01,431


## EVALUAR

### Predecir sobre validación

In [11]:
pred = modelo.best_estimator_.predict(val_x)

### Evaluar sobre validación

In [12]:
roc_auc_score(val_y, pred)

np.float64(0.8107583795943246)

### Examinar el mejor modelo

In [13]:
modelo.best_estimator_

Pipeline(steps=[('algoritmo',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.01,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=1000, n_jobs=-1,
                               num_parallel_tree=None, ...))])

In [14]:
ruta_modelo = r"C:\Users\DataScience\ONLINE_DS_THEBRIDGE_DaniCastillo\ML_LEAD SCORING\01_src\06_models\modelo_xgboost.pkl"

# Guardar el modelo en formato pickle
with open(ruta_modelo, 'wb') as archivo:
    pickle.dump(modelo.best_estimator_, archivo)

print(f"✅ Modelo guardado en: {ruta_modelo}")


✅ Modelo guardado en: C:\Users\DataScience\ONLINE_DS_THEBRIDGE_DaniCastillo\ML_LEAD SCORING\01_src\06_models\modelo_xgboost.pkl
